# マイクロマウスのターン生成

## 概要
マイクロマウス競技において、高速な走行を行うためにはスラロームと呼ばれる車体を止めずにターンをする動作が必要とされます。
このプログラムでは、スラローム走行に必要な軌跡を生成します。
    

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt 
import matplotlib.ticker as tick
import numpy as np 
from numba import jit
import math 
from itertools import product

from ipywidgets import interact, interactive, fixed


In [2]:
# molifireの積分値を計算するための関数


#y = math.e / a * np.exp( (-1)/(1- np.abs(2*x/b -1)**c))

@jit('f8(f8, f8, f8, f8)')
def moli(x,a,b,c):
    if c <= 0.0:
        return 0.0    
    if x <= 0.0:
        return 0.0
    elif x >= b:        
        return 0.0
    else:
        return math.e / a * math.exp( (-1)/(1- abs(2*x/b -1)**c))

@jit('f8(f8)')
def int_moli(c):
    delta_t = 0.000001
    x_list = np.arange(0.0,1.0,delta_t)

    S = 0.0                
    for x in x_list:
        y = moli(x, 1.0, 1.0, c)
        S = S + delta_t * y
    return S    


In [3]:
for i in np.arange(0.0, 6.01, 0.01):    
    print("{:.9f}".format(int_moli(i))+"f,")

0.000000000f,
0.000001017f,
0.000006619f,
0.000069708f,
0.000302330f,
0.000821610f,
0.001712566f,
0.003022231f,
0.004766678f,
0.006940295f,
0.009523647f,
0.012489283f,
0.015805719f,
0.019440060f,
0.023359661f,
0.027533130f,
0.031930903f,
0.036525520f,
0.041291734f,
0.046206494f,
0.051248864f,
0.056399903f,
0.061642527f,
0.066961357f,
0.072342575f,
0.077773783f,
0.083243869f,
0.088742883f,
0.094261924f,
0.099793036f,
0.105329113f,
0.110863812f,
0.116391482f,
0.121907087f,
0.127406149f,
0.132884689f,
0.138339180f,
0.143766498f,
0.149163885f,
0.154528909f,
0.159859436f,
0.165153595f,
0.170409757f,
0.175626508f,
0.180802629f,
0.185937077f,
0.191028964f,
0.196077550f,
0.201082218f,
0.206042471f,
0.210957914f,
0.215828246f,
0.220653253f,
0.225432796f,
0.230166802f,
0.234855264f,
0.239498228f,
0.244095789f,
0.248648089f,
0.253155308f,
0.257617661f,
0.262035398f,
0.266408793f,
0.270738149f,
0.275023788f,
0.279266054f,
0.283465307f,
0.287621923f,
0.291736289f,
0.295808805f,
0.299839882f,
0.3038

In [4]:

# 軌跡を作るためのクラス
class CalcTrajWithBeta:
    shape_factor = 0
    path_length = 0
    target_ang = 0
    delta_t = 0
    tread = 0
    start_ang = 0
    start_x = 0
    start_y = 0
    v = 0
    cp = 0    
    
    t = None
    omega = None
    alpha = None
    ang   = None    
    c_traj_x = None
    c_traj_y = None
    l_traj_x = None
    l_traj_y = None
    r_traj_x = None
    r_traj_y = None

    t_beta = None
    omega_beta = None
    alpha_beta = None
    ang_beta   = None        
    beta  = None    
    
    c_traj_x_beta = None
    c_traj_y_beta = None
    l_traj_x_beta = None
    l_traj_y_beta = None
    r_traj_x_beta = None
    r_traj_y_beta = None

    
    
    # 軌跡のパラメータをセットする
    def set_param(self, shape_factor_, path_length_, target_ang_, delta_t_, tread_, v_, cp_,start_x_=0, start_y_=0, start_ang_=0):
        self.shape_factor = shape_factor_
        self.path_length = path_length_
        self.target_ang = target_ang_
        self.delta_t = delta_t_
        self.tread = tread_
        self.start_x = start_x_
        self.start_y = start_y_
        self.start_ang = start_ang_
        self.v= v_
        self.cp = cp_
        
    # 軌跡を計算する
        
    def calc_traj(self):        
        path_end_time = self.path_length/ self.v
        self.t = np.arange(0.0, path_end_time, self.delta_t)
        
        # a * b * S = target_ang
        c = self.shape_factor
        b = self.path_length
        S = int_moli(c)
        a = 1/(self.target_ang/b/S)
                
        # 角速度
        self.omega = np.zeros(len(self.t),dtype=np.float64)
        for i in range(len(self.t)):
            self.omega[i] = self.v * moli(self.t[i]*self.v, a, b, c)                
        
        # 角速度と時間のリストから角速度が限りなく0に近い部分をトリミング
        index = 0
        del_count = 0
        while index < len(self.omega):
            if self.omega[index] < 0.01745: # 0.01745 rad/s = 1deg/s
                self.omega = np.delete(self.omega, index)
                self.t     = np.delete(self.t, index)
                path_end_time = path_end_time - self.delta_t
                index = 0
                del_count = del_count + 1                
            else:
                index = index + 1
        #print ("del_count", del_count)
        
        # 角加速度
        self.alpha = np.gradient(self.omega) / self.delta_t
        
        # 角度
        self.ang = np.zeros(len(self.omega))
        self.ang[0] = self.start_ang;
        for i,ele in enumerate(self.omega):
            if i != 0:
                self.ang[i] = self.ang[i-1] + self.delta_t * ele 
                
        # スリップ角
        self.t_beta = self.t.copy()
        self.omega_beta = self.omega.copy()
        self.alpha_beta = self.alpha.copy()
        self.ang_beta   = self.ang.copy()
        self.beta = np.zeros(len(self.omega))
        
        def f(t,y):
            return -self.cp * y/self.v - self.v * moli(t*self.v, a, b, c)
                
        i = 0
        slip_after_count = 0
        while i < len(self.omega_beta):
            if i != 0:
                k1 = f(self.t_beta[i-1], self.beta[i-1])
                k2 = f(self.t_beta[i-1]+self.delta_t*0.5, self.beta[i-1]+self.delta_t*0.5*k1)
                k3 = f(self.t_beta[i-1]+self.delta_t*0.5, self.beta[i-1]+self.delta_t*0.5*k2)
                k4 = f(self.t_beta[i-1]+self.delta_t, self.beta[i-1]+self.delta_t*k3)
                               
                self.beta[i] = self.beta[i-1] + (k1+2*k2+2*k3+k4)* self.delta_t /6.0                        

            if i == len(self.omega_beta)-1 and abs(self.beta[i]) > 0.01745:
                slip_after_count = slip_after_count + 1
                self.t_beta = np.append(self.t_beta, self.t_beta[-1]+self.delta_t)
                self.omega_beta = np.append(self.omega_beta, 0.0)
                self.alpha_beta = np.append(self.alpha_beta, 0.0)
                self.ang_beta = np.append(self.ang_beta, self.ang_beta[-1])
                self.beta = np.append(self.beta, 0.0)                              
            i = i + 1
        #print("slip_after_count:", slip_after_count)
        
        # スリップ角考慮なしの軌跡を求める       
        self.c_traj_x = np.zeros(len(self.omega))
        self.c_traj_y = np.zeros(len(self.omega))        
        self.l_traj_x = np.zeros(len(self.omega))
        self.l_traj_y = np.zeros(len(self.omega))
        self.r_traj_x = np.zeros(len(self.omega))
        self.r_traj_y = np.zeros(len(self.omega))
        self.c_traj_x[0] = self.start_x
        self.c_traj_y[0] = self.start_y        
        self.r_traj_x[0] = self.c_traj_x[0] + math.sin(self.start_ang)*self.tread/2
        self.r_traj_y[0] = self.c_traj_y[0] - math.cos(self.start_ang)*self.tread/2
        self.l_traj_x[0] = self.c_traj_x[0] - math.sin(self.start_ang)*self.tread/2
        self.l_traj_y[0] = self.c_traj_y[0] + math.cos(self.start_ang)*self.tread/2
        
        for i,ele in enumerate(self.ang):
            if i != 0:
                
                if self.omega[i] > 0.087: # 0.087rad = 5deg
                    self.c_traj_x[i] = self.c_traj_x[i-1] + self.v * math.cos(ele) * math.sin(self.omega[i]* self.delta_t * 0.5) / (self.omega[i] * 0.5)
                    self.c_traj_y[i] = self.c_traj_y[i-1] + self.v * math.sin(ele) * math.sin(self.omega[i]* self.delta_t * 0.5) / (self.omega[i] * 0.5)
                else:
                    self.c_traj_x[i] = self.c_traj_x[i-1] + self.v * math.cos(ele) * self.delta_t
                    self.c_traj_y[i] = self.c_traj_y[i-1] + self.v * math.sin(ele) * self.delta_t
                                    
                self.r_traj_x[i] = self.c_traj_x[i] + math.sin(ele)*self.tread/2
                self.r_traj_y[i] = self.c_traj_y[i] - math.cos(ele)*self.tread/2

                self.l_traj_x[i] = self.c_traj_x[i] - math.sin(ele)*self.tread/2
                self.l_traj_y[i] = self.c_traj_y[i] + math.cos(ele)*self.tread/2

        # スリップ角考慮ありの軌跡を求める
        self.c_traj_x_beta = np.zeros(len(self.omega_beta))
        self.c_traj_y_beta = np.zeros(len(self.omega_beta))        
        self.l_traj_x_beta = np.zeros(len(self.omega_beta))
        self.l_traj_y_beta = np.zeros(len(self.omega_beta))
        self.r_traj_x_beta = np.zeros(len(self.omega_beta))
        self.r_traj_y_beta = np.zeros(len(self.omega_beta))
        self.c_traj_x_beta[0] = self.start_x
        self.c_traj_y_beta[0] = self.start_y        
        self.r_traj_x_beta[0] = self.c_traj_x_beta[0] + math.sin(self.start_ang)*self.tread/2
        self.r_traj_y_beta[0] = self.c_traj_y_beta[0] - math.cos(self.start_ang)*self.tread/2
        self.l_traj_x_beta[0] = self.c_traj_x_beta[0] - math.sin(self.start_ang)*self.tread/2
        self.l_traj_y_beta[0] = self.c_traj_y_beta[0] + math.cos(self.start_ang)*self.tread/2
                
        for i,ele in enumerate(self.ang_beta):
            if i != 0:                
                # スリップ角考慮あり曲線
                if self.omega_beta[i] > 0.087: # 0.087rad = 5deg
                    self.c_traj_x_beta[i] = self.c_traj_x_beta[i-1] + self.v * math.cos(ele + self.beta[i]) * math.sin(self.omega[i]* self.delta_t * 0.5) / (self.omega[i] * 0.5)
                    self.c_traj_y_beta[i] = self.c_traj_y_beta[i-1] + self.v * math.sin(ele + self.beta[i]) * math.sin(self.omega[i]* self.delta_t * 0.5) / (self.omega[i] * 0.5)
                else:
                    self.c_traj_x_beta[i] = self.c_traj_x_beta[i-1] + self.v * math.cos(ele + self.beta[i]) * self.delta_t
                    self.c_traj_y_beta[i] = self.c_traj_y_beta[i-1] + self.v * math.sin(ele + self.beta[i]) * self.delta_t

                self.r_traj_x_beta[i] = self.c_traj_x_beta[i] + math.sin(ele)*self.tread/2
                self.r_traj_y_beta[i] = self.c_traj_y_beta[i] - math.cos(ele)*self.tread/2

                self.l_traj_x_beta[i] = self.c_traj_x_beta[i] - math.sin(ele)*self.tread/2
                self.l_traj_y_beta[i] = self.c_traj_y_beta[i] + math.cos(ele)*self.tread/2                
            
    # 軌跡が終わったポイントのx座標
    def get_end_x(self):
        return self.c_traj_x[len(self.c_traj_x)-1]
    # 軌跡が終わったポイントのy座標    
    def get_end_y(self):
        return self.c_traj_y[len(self.c_traj_y)-1]
    
    # x方向の移動距離
    def get_move_x(self):
        return self.c_traj_x[len(self.c_traj_x)-1] - self.start_x
    
    # y方向の移動距離
    def get_move_y(self):
        return self.c_traj_y[len(self.c_traj_y)-1] - self.start_y
        
    # 速度一定で走ったときの時間と角速度のリストを返す
    def get_omega(self):
        return self.t, self.omega
    
    # 速度一定で走ったときの時間と角速度(degree)のリストを返す
    def get_omega_degree(self):
        t, rad = self.get_omega() 
        return t, rad * 180 /math.pi
    
    # 速度一定で走ったときの時間と角加速度のリストを返す
    def get_alpha(self):
        return self.t,self.alpha 

    # 速度一定で走ったときの時間と角加速度(degree)のリストを返す
    def get_alpha_degree(self):
        t, alpha = self.get_alpha()
        return t, alpha * 180/math.pi

    # 速度一定で走ったときの時間と角度のリストを返す
    def get_ang(self):        
        return self.t, self.ang    
    
    # 速度一定で走ったときの時間と角度(degree)のリストを返す
    def get_ang_degree(self):
        t, ang = self.get_ang()
        return t, ang * 180/math.pi
        
    # 速度一定で走ったときの時間と向進加速度のリストを返す
    def get_cent_acc(self):
        return self.t, self.v * self.omega 

    # 速度一定で走ったときの時間とスリップ角のリストを返す
    def get_beta(self):
        return self.t_beta, self.beta
    
    # 速度一定で走ったときの時間とスリップ角(degree)のリストを返す
    def get_beta_degree(self):
        t, rad = self.get_beta() 
        return t, rad * 180 /math.pi

    # スリップ角有の軌跡が終わったポイントのx座標
    def get_end_x_beta(self):
        return self.c_traj_x_beta[-1]
    
    # スリップ角有の軌跡が終わったポイントのy座標    
    def get_end_y_beta(self):
        return self.c_traj_y_beta[-1]
    
    # スリップ角有のx方向の移動距離
    def get_move_x_beta(self):
        return self.c_traj_x_beta[-1] - self.start_x
    
    # スリップ角有のy方向の移動距離
    def get_move_y_beta(self):
        return self.c_traj_y_beta[-1] - self.start_y
        

In [5]:
def simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v):
    traj_curve = CalcTrajWithBeta()
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, v_=v, cp_ = cp, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()

    # 前後距離の算出
    X = end_x - start_x - traj_curve.get_move_x_beta()
    Y = end_y - start_y - traj_curve.get_move_y_beta()
    end_ang = start_ang + target_ang
    sin_start_ang = math.sin(start_ang * math.pi/180.0)
    cos_start_ang = math.cos(start_ang * math.pi/180.0)
    sin_end_ang = math.sin(end_ang * math.pi/180.0)
    cos_end_ang = math.cos(end_ang * math.pi/180.0)
        
    d_pre_beta = (sin_end_ang   * X - cos_end_ang   * Y) / (cos_start_ang * sin_end_ang   - cos_end_ang   * sin_start_ang)
    d_fol_beta = (sin_start_ang * X - cos_start_ang * Y) / (cos_end_ang   * sin_start_ang - cos_start_ang * sin_end_ang)
    
    # 値格納用変数の初期化
    t, omega = traj_curve.get_omega_degree()
    t, alpha = traj_curve.get_alpha_degree()
    t, ang   = traj_curve.get_ang_degree()
    t, cent_acc = traj_curve.get_cent_acc()
    t_beta, beta  = traj_curve.get_beta_degree()    
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([traj_curve.r_traj_x_beta[0],traj_curve.r_traj_x_beta[0] + d_pre_beta*cos_start_ang], \
             [traj_curve.r_traj_y_beta[0],traj_curve.r_traj_y_beta[0] + d_pre_beta*sin_start_ang], color="blue") 
    
    plt.plot([start_x, traj_curve.c_traj_x_beta[0] + d_pre_beta * cos_start_ang], \
             [start_y, traj_curve.c_traj_y_beta[0] + d_pre_beta * sin_start_ang], color="blue") 
    
    plt.plot([traj_curve.l_traj_x_beta[0], traj_curve.l_traj_x_beta[0] + d_pre_beta*cos_start_ang], \
             [traj_curve.l_traj_y_beta[0], traj_curve.l_traj_y_beta[0] + d_pre_beta*sin_start_ang], color="blue") 
        
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.r_traj_x_beta + d_pre_beta*cos_start_ang, \
             traj_curve.r_traj_y_beta + d_pre_beta*sin_start_ang, color="orange")
    
    plt.plot(traj_curve.c_traj_x_beta + d_pre_beta*cos_start_ang, 
             traj_curve.c_traj_y_beta + d_pre_beta*sin_start_ang, color="orange")
    
    plt.plot(traj_curve.l_traj_x_beta + d_pre_beta*cos_start_ang, \
             traj_curve.l_traj_y_beta + d_pre_beta*sin_start_ang, color="orange")
        
    # 直後部分の軌跡描画
    sin_end_p90_ang = math.sin((end_ang+90) * math.pi/180.0)
    cos_end_p90_ang = math.cos((end_ang+90) * math.pi/180.0)
    sin_end_m90_ang = math.sin((end_ang-90) * math.pi/180.0)
    cos_end_m90_ang = math.cos((end_ang-90) * math.pi/180.0)
    
    plt.plot([traj_curve.get_end_x_beta() + d_pre_beta*cos_start_ang + tread/2*cos_end_p90_ang, end_x+tread/2*cos_end_p90_ang], \
             [traj_curve.get_end_y_beta() + d_pre_beta*sin_start_ang + tread/2*sin_end_p90_ang, end_y+tread/2*sin_end_p90_ang], color="blue") 

    plt.plot([traj_curve.get_end_x_beta() + d_pre_beta*cos_start_ang, end_x], \
             [traj_curve.get_end_y_beta() + d_pre_beta*sin_start_ang, end_y], color="blue")

    plt.plot([traj_curve.get_end_x_beta() + d_pre_beta*cos_start_ang + tread/2*cos_end_m90_ang, end_x+tread/2*cos_end_m90_ang], \
             [traj_curve.get_end_y_beta() + d_pre_beta*sin_start_ang + tread/2*sin_end_m90_ang, end_y+tread/2*sin_end_m90_ang], color="blue")
                        
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.title("trajectory")
    plt.axis([-0.18, 0.18, -0.18, 0.18])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.09+ 0.045-0.003, j*0.09+0.045-0.003], width=0.006, height=0.006) )
    plt.show()

      
    #print("d_pre x[m] " + str(d_pre))
    #print("d_fol y[m] " + str(d_fol))
    #print("total length[m] " + str(d_pre+d_fol+path_length__) )
    print("d_pre_beta[m] " + str(d_pre_beta))
    print("d_fol_beta[m] " + str(d_fol_beta))
    print("total length[m] " + str(d_pre_beta+d_fol_beta+traj_curve.t_beta[-1] * v) )

    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.xlim(t_beta[0], t_beta[-1])
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.title("beta [deg] ")
    plt.grid()
    plt.xlim(t_beta[0], t_beta[-1])
    plt.plot(t_beta,-beta, label="beta", color="red")
        
    plt.subplot(4, 1, 3)
    plt.plot(t,alpha)
    plt.xlim(t_beta[0], t_beta[-1])
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.xlim(t_beta[0], t_beta[-1])
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()


def simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v):
    traj_curve = CalcTrajWithBeta()
    traj_curve.set_param(shape_factor_=shape_factor, path_length_= path_length, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, v_=v, cp_ = cp, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)
    traj_curve.calc_traj()

    # 前後距離の算出
    X = end_x - start_x - traj_curve.get_move_x_beta()
    Y = end_y - start_y - traj_curve.get_move_y_beta()
    end_ang = start_ang + target_ang
    sin_start_ang = math.sin(start_ang * math.pi/180.0)
    cos_start_ang = math.cos(start_ang * math.pi/180.0)
    sin_end_ang = math.sin(end_ang * math.pi/180.0)
    cos_end_ang = math.cos(end_ang * math.pi/180.0)
        
    d_pre_beta = (sin_end_ang   * X - cos_end_ang   * Y) / (cos_start_ang * sin_end_ang   - cos_end_ang   * sin_start_ang)
    d_fol_beta = (sin_start_ang * X - cos_start_ang * Y) / (cos_end_ang   * sin_start_ang - cos_start_ang * sin_end_ang)
    
    # 値格納用変数の初期化
    t, omega = traj_curve.get_omega_degree()
    t, alpha = traj_curve.get_alpha_degree()
    t, ang   = traj_curve.get_ang_degree()
    t, cent_acc = traj_curve.get_cent_acc()
    t_beta, beta  = traj_curve.get_beta_degree()    
    
    return (d_pre_beta, d_fol_beta, np.max(np.abs(beta)))

In [8]:
def simulate180(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length_min, path_length_max, v):
    traj_curve = CalcTrajWithBeta()    
    error_y = 1.0
    
    # 最小化するターゲット関数
    def calc_y_error(pl_):
        traj_curve = CalcTrajWithBeta()
        traj_curve.set_param(shape_factor_=shape_factor, path_length_= pl_, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, v_=v, cp_ = cp, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)

        traj_curve.calc_traj()
        
        error_y_ = traj_curve.get_move_y_beta() - end_y
        return error_y_, traj_curve
    
    while abs(error_y) > 0.0001:
        pl = 0.5 * (path_length_min + path_length_max)
        error_y, traj_curve = calc_y_error(pl)
        path_length = pl
        if(error_y > 0):
            path_length_max = pl
        else:
            path_length_min = pl
        print("error_y: " + str(error_y))
        print("path_length: " + str(pl))
        print("min, max :",path_length_min, path_length_max)
 
    # 前後距離の算出
    X = end_x - start_x - traj_curve.get_move_x_beta()
    Y = end_y - start_y - traj_curve.get_move_y_beta()
    end_ang = start_ang + target_ang
    sin_start_ang = math.sin(start_ang * math.pi/180.0)
    cos_start_ang = math.cos(start_ang * math.pi/180.0)
    sin_end_ang = math.sin(end_ang * math.pi/180.0)
    cos_end_ang = math.cos(end_ang * math.pi/180.0)
        
    #d_pre_beta = (sin_end_ang   * X - cos_end_ang   * Y) / (cos_start_ang * sin_end_ang   - cos_end_ang   * sin_start_ang)
    #d_fol_beta = (sin_start_ang * X - cos_start_ang * Y) / (cos_end_ang   * sin_start_ang - cos_start_ang * sin_end_ang)
    d_pre_beta =0.027
    d_fol_beta = d_pre_beta + traj_curve.get_move_x_beta()     
    
    # 値格納用変数の初期化
    t, omega = traj_curve.get_omega_degree()
    t, alpha = traj_curve.get_alpha_degree()
    t, ang   = traj_curve.get_ang_degree()
    t, cent_acc = traj_curve.get_cent_acc()
    t_beta, beta  = traj_curve.get_beta_degree()    
    plt.axes().set_aspect('equal')
    
    # 前直線部分の軌跡描画
    plt.plot([start_x,start_x + d_pre_beta],[start_y +tread/2 , start_y+tread/2]  ,color="red") 
    plt.plot([start_x,start_x + d_pre_beta],[start_y, start_y]  ,color="red") 
    plt.plot([start_x,start_x + d_pre_beta],[start_y -tread/2, start_y-tread/2  ]  ,color="red") 
    
    # カーブ部分の軌跡描画
    plt.plot(traj_curve.c_traj_x_beta + d_pre_beta, traj_curve.c_traj_y_beta, color="blue")
    plt.plot(traj_curve.r_traj_x_beta + d_pre_beta, traj_curve.r_traj_y_beta, color="blue")
    plt.plot(traj_curve.l_traj_x_beta + d_pre_beta, traj_curve.l_traj_y_beta, color="blue")

    # 直後部分の軌跡描画
    plt.plot([traj_curve.get_end_x_beta()+ d_pre_beta , end_x],[traj_curve.get_end_y_beta()+tread/2, traj_curve.get_end_y_beta()+tread/2]  ,color="red") 
    plt.plot([traj_curve.get_end_x_beta()+ d_pre_beta , end_x],[traj_curve.get_end_y_beta(), traj_curve.get_end_y_beta()]  ,color="red") 
    plt.plot([traj_curve.get_end_x_beta()+ d_pre_beta , end_x],[traj_curve.get_end_y_beta()-tread/2, traj_curve.get_end_y_beta()-tread/2]  ,color="red") 
               
    plt.gca().xaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.gca().yaxis.set_major_locator(tick.MultipleLocator(0.045))
    plt.title("trajectory")
    plt.axis([-0.18, 0.18, -0.18, 0.18])

    plt.grid(True)
    for i in range(-2,4):
        for j in range(-2,4):
            plt.gca().add_patch( plt.Rectangle(xy=[i*0.09+ 0.045-0.003, j*0.09+0.045-0.003], width=0.006, height=0.006) )
    plt.show()

      
    #print("d_pre x[m] " + str(d_pre))
    #print("d_fol y[m] " + str(d_fol))
    #print("total length[m] " + str(d_pre+d_fol+path_length__) )
    print("d_pre_beta x[m] " + str(d_pre_beta))
    print("d_fol_beta y[m] " + str(d_fol_beta))
    print("total length[m] " + str(d_pre_beta+d_fol_beta+traj_curve.t_beta[-1] * v) )

    
    plt.subplot(4, 1, 1)
    plt.plot(t,omega)
    plt.xlim(t_beta[0], t_beta[-1])
    plt.title("omega [sec - deg/s] ")
    plt.grid()  

    plt.subplot(4, 1, 2)
    plt.title("beta [deg] ")
    plt.grid()
    plt.xlim(t_beta[0], t_beta[-1])
    plt.plot(t_beta,-beta, label="beta", color="red")
        
    plt.subplot(4, 1, 3)
    plt.plot(t,alpha)
    plt.xlim(t_beta[0], t_beta[-1])
    plt.title("alpha [sec - deg/ss] ")
    plt.grid()

    plt.subplot(4, 1, 4)
    plt.title("centripetal acc [sec - m/ss] ")
    plt.xlim(t_beta[0], t_beta[-1])
    plt.grid()
    plt.plot(t,cent_acc, label="centripetal acc")
    
    plt.tight_layout() 
    plt.show()

def simulate180_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length_min, path_length_max, v):
    traj_curve = CalcTrajWithBeta()    
    error_y = 1.0
    
    # 最小化するターゲット関数
    def calc_y_error(pl_):
        traj_curve = CalcTrajWithBeta()
        traj_curve.set_param(shape_factor_=shape_factor, path_length_= pl_, target_ang_= target_ang* math.pi/180.0 , 
                   delta_t_=delta_t, tread_=tread, v_=v, cp_ = cp, start_x_=start_x, start_y_=start_y, start_ang_=start_ang* math.pi/180.0)

        traj_curve.calc_traj()
        
        error_y_ = traj_curve.get_move_y_beta() - end_y
        return error_y_, traj_curve
    
    while abs(error_y) > 0.0001:
        pl = 0.5 * (path_length_min + path_length_max)
        error_y, traj_curve = calc_y_error(pl)
        path_length = pl
        if(error_y > 0):
            path_length_max = pl
        else:
            path_length_min = pl
 
    # 前後距離の算出
    X = end_x - start_x - traj_curve.get_move_x_beta()
    Y = end_y - start_y - traj_curve.get_move_y_beta()
    end_ang = start_ang + target_ang
    sin_start_ang = math.sin(start_ang * math.pi/180.0)
    cos_start_ang = math.cos(start_ang * math.pi/180.0)
    sin_end_ang = math.sin(end_ang * math.pi/180.0)
    cos_end_ang = math.cos(end_ang * math.pi/180.0)
        
    #d_pre_beta = (sin_end_ang   * X - cos_end_ang   * Y) / (cos_start_ang * sin_end_ang   - cos_end_ang   * sin_start_ang)
    #d_fol_beta = (sin_start_ang * X - cos_start_ang * Y) / (cos_end_ang   * sin_start_ang - cos_start_ang * sin_end_ang)
    d_pre_beta =0.027
    d_fol_beta = d_pre_beta + traj_curve.get_move_x_beta()     
    
    # 値格納用変数の初期化
    t, omega = traj_curve.get_omega_degree()
    t, alpha = traj_curve.get_alpha_degree()
    t, ang   = traj_curve.get_ang_degree()
    t, cent_acc = traj_curve.get_cent_acc()
    t_beta, beta  = traj_curve.get_beta_degree()        
    
    return (d_pre_beta, d_fol_beta, np.max(np.abs(beta)))

## ハーフ90度ターン


In [11]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.045
start_y = 0.00
end_x = 0.0
end_y = 0.045
target_ang = 90
start_ang = 0.0
cp = 100

shape_factor__ = 4
path_length__ = 0.06
v__ = 0.6

print("v,   pre,    fol,    beta_max")
for v in np.arange(0.1, 1.1, 0.1):
    _pre, _fol, _beta_max = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:.2f}'.format(v), '{:.5f}'.format(_pre), '{:.5f}'.format(_fol), '{:.1f}'.format(_beta_max))



@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)

v,   pre,    fol,    beta_max
0.10 0.00975 0.00975 0.2
0.20 0.00945 0.00985 0.8
0.30 0.00904 0.01024 1.8
0.40 0.00841 0.01080 3.1
0.50 0.00737 0.01084 4.9
0.60 0.00648 0.00942 7.1
0.70 0.00533 0.00644 9.6
0.80 0.00391 0.00297 12.4
0.90 0.00219 -0.00188 15.6
1.00 -0.00075 -0.00810 18.9


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

## ハーフ大周り90度

In [12]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.09
start_y = 0.00
end_x = 0.0
end_y = 0.09
target_ang = 90
start_ang = 0.0
cp = 100

shape_factor__ = 4
path_length__ = 0.11
v__ = 0.5

print("v,   pre,    fol,    beta_max")
for v in np.arange(0.1, 1.1, 0.1):
    _pre, _fol, _beta_max = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:.2f}'.format(v), '{:.5f}'.format(_pre), '{:.5f}'.format(_fol), '{:.1f}'.format(_beta_max))


@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


v,   pre,    fol,    beta_max
0.10 0.02569 0.02570 0.1
0.20 0.02519 0.02560 0.4
0.30 0.02478 0.02589 1.0
0.40 0.02417 0.02657 1.7
0.50 0.02294 0.02693 2.7
0.60 0.02230 0.02787 3.9
0.70 0.02074 0.02776 5.3
0.80 0.01952 0.02541 6.9
0.90 0.01804 0.02169 8.7
1.00 0.01531 0.01708 10.7


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

## ハーフ180度ターン
180度ターンは曲線長と角度を指定するだけでうまく終点と始点が結べないため注意。
カーブの終点が180度ターンの終了点になるように曲線長を2分探索で求めている。

In [13]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = 0.00
start_y = 0.00
end_x = 0.0
end_y = 0.09
target_ang = 180
start_ang = 0.0
cp = 100

shape_factor__ = 3.6
path_length__ = 0.1
v__ = 0.5

path_length_min = 0.01
path_length_max = 0.5

print("v,   pre,    fol,    beta_max")
for v in np.arange(0.1, 1.1, 0.1):
    _pre, _fol, _beta_max = simulate180_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length_min, path_length_max, v)
    print('{:.2f}'.format(v), '{:.5f}'.format(_pre), '{:.5f}'.format(_fol), '{:.1f}'.format(_beta_max))



@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate180(tread, delta_t, start_x, start_y, end_x, end_y, target_ang, start_ang, cp, shape_factor, path_length_min, path_length_max, v)



v,   pre,    fol,    beta_max
0.10 0.02700 0.02702 0.1
0.20 0.02700 0.02731 0.5
0.30 0.02700 0.02831 1.2
0.40 0.02700 0.02943 2.1
0.50 0.02700 0.03083 3.4
0.60 0.02700 0.03335 4.8
0.70 0.02700 0.03536 6.6
0.80 0.02700 0.03516 8.6
0.90 0.02700 0.03387 11.0
1.00 0.02700 0.03136 13.7


interactive(children=(FloatSlider(value=3.6, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

## ハーフ 斜めS2D 45度ターン

In [20]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.09
start_y = 0.00
end_x = 0.0
end_y = 0.045
target_ang = 45
start_ang = 0.0
cp = 100

shape_factor__ = 4
path_length__ = 0.04
v__ = 0.5

print("v,   pre,    fol,    beta_max")
for v in np.arange(0.1, 1.1, 0.1):
    _pre, _fol, _beta_max = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:.2f}'.format(v), '{:.5f}'.format(_pre), '{:.5f}'.format(_fol), '{:.1f}'.format(_beta_max))


@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


v,   pre,    fol,    beta_max
0.10 0.02502 0.04367 0.1
0.20 0.02472 0.04376 0.6
0.30 0.02442 0.04436 1.3
0.40 0.02373 0.04474 2.4
0.50 0.02313 0.04470 3.7
0.60 0.02231 0.04306 5.3
0.70 0.02061 0.04057 7.1
0.80 0.01929 0.03767 9.1
0.90 0.01774 0.03374 11.2
1.00 0.01596 0.02821 13.4


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

## ハーフ 斜めS2D 135度ターン

In [16]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.09
start_y = 0.00
end_x = -0.045
end_y = 0.09
target_ang = 135
start_ang = 0.0
cp = 100

shape_factor__ = 4
path_length__ = 0.11
v__ = 0.5

print("v,   pre,    fol,    beta_max")
for v in np.arange(0.1, 1.1, 0.1):
    _pre, _fol, _beta_max = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:.2f}'.format(v), '{:.5f}'.format(_pre), '{:.5f}'.format(_fol), '{:.1f}'.format(_beta_max))


@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


v,   pre,    fol,    beta_max
0.10 0.03646 0.02875 0.2
0.20 0.03596 0.02864 0.6
0.30 0.03543 0.02911 1.4
0.40 0.03467 0.02935 2.6
0.50 0.03375 0.03003 4.0
0.60 0.03297 0.03083 5.8
0.70 0.03113 0.02836 7.9
0.80 0.02952 0.02531 10.3
0.90 0.02660 0.02071 13.0
1.00 0.02400 0.01403 16.0


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

## ハーフ 斜め 90度ターン

In [17]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.045
start_y = 0.00
end_x = -0.045
end_y = 0.09
target_ang = 90
start_ang = 45
cp = 100

print("v,   pre,    fol,    beta_max")
for v in np.arange(0.1, 1.1, 0.1):
    _pre, _fol, _beta_max = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:.2f}'.format(v), '{:.5f}'.format(_pre), '{:.5f}'.format(_fol), '{:.1f}'.format(_beta_max))


shape_factor__ = 4
path_length__ = 0.07
v__ = 0.5

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


v,   pre,    fol,    beta_max
0.10 -0.00067 -0.00066 0.1
0.20 -0.00117 -0.00076 0.4
0.30 -0.00158 -0.00047 1.0
0.40 -0.00219 0.00021 1.7
0.50 -0.00342 0.00057 2.7
0.60 -0.00406 0.00151 3.9
0.70 -0.00562 0.00140 5.3
0.80 -0.00684 -0.00095 6.9
0.90 -0.00833 -0.00467 8.7
1.00 -0.01105 -0.00928 10.7


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

# ハーフ斜め D2S 45度ターン

In [18]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.045
start_y = 0.0
end_x = 0.0
end_y = 0.09
target_ang = 45
start_ang = 45
cp = 100

print("v,   pre,    fol,    beta_max")
for v in np.arange(0.1, 1.1, 0.1):
    _pre, _fol, _beta_max = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:.2f}'.format(v), '{:.5f}'.format(_pre), '{:.5f}'.format(_fol), '{:.1f}'.format(_beta_max))


shape_factor__ = 4
path_length__ = 0.04
v__ = 0.5

@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


v,   pre,    fol,    beta_max
0.10 0.02890 0.01027 0.1
0.20 0.02850 0.01027 0.3
0.30 0.02800 0.01067 0.8
0.40 0.02750 0.01126 1.3
0.50 0.02661 0.01193 2.1
0.60 0.02593 0.01299 3.0
0.70 0.02434 0.01182 4.1
0.80 0.02313 0.00945 5.4
0.90 0.02169 0.00617 6.7
1.00 0.02004 0.00207 8.2


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…

# ハーフ斜め D2S 135度ターン

In [19]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,8

# 設定パラメータ
tread = 0.038
delta_t = 0.001
start_x = -0.045
start_y = 0.00
end_x = -0.09
end_y = 0.09
target_ang = 135
start_ang = 45
cp = 100

shape_factor__ = 4
path_length__ = 0.11
v__ = 0.5

print("v,   pre,    fol,    beta_max")
for v in np.arange(0.1, 1.1, 0.1):
    _pre, _fol, _beta_max = simulate_without_plot(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor__, path_length__, v)
    print('{:.2f}'.format(v), '{:.5f}'.format(_pre), '{:.5f}'.format(_fol), '{:.1f}'.format(_beta_max))



@interact(shape_factor=(0.1, 8.0, 0.1), path_length=(0.01, 0.36, 0.01),  v = (0.01, 1.0, 0.1)  )
def h(shape_factor=shape_factor__, path_length=path_length__, v=v__):
    simulate(tread, delta_t, start_x, start_y, end_x, end_y, target_ang,  start_ang, cp, shape_factor, path_length, v)


v,   pre,    fol,    beta_max
0.10 0.02874 0.03647 0.2
0.20 0.02823 0.03636 0.6
0.30 0.02771 0.03683 1.4
0.40 0.02695 0.03707 2.6
0.50 0.02603 0.03775 4.0
0.60 0.02525 0.03855 5.8
0.70 0.02341 0.03608 7.9
0.80 0.02180 0.03303 10.3
0.90 0.01888 0.02844 13.0
1.00 0.01628 0.02175 16.0


interactive(children=(FloatSlider(value=4.0, description='shape_factor', max=8.0, min=0.1), FloatSlider(value=…